# Comparison b/w RNN & SVR

In [ ]:
# ------------------------------------------------------------------------------
# Imports
# ------------------------------------------------------------------------------
!pip3 install https://github.com/google-coral/pycoral/releases/download/release-frogfish/tflite_runtime-2.5.0-cp36-cp36m-linux_x86_64.whl

from loadmat import loadmat
import numpy as np
from scipy.signal import decimate

from tensorflow import keras

%matplotlib notebook
import matplotlib.pyplot as plt

from timeit import default_timer as timer

import prediction_model

In [ ]:
test_files  = ["calibration_struct{:d}.mat".format(i) for i in [2, 4, 6, 7, 5]]
n_history = 5

# ------------------------------------------------------------------------------
# Lambda functions for data extractions
# ------------------------------------------------------------------------------
get_X = lambda tmp: np.vstack((
    tmp["Fx_measured"],
    tmp["Fy_measured"],
    tmp["Fz_measured"],
    tmp["Mx_measured"],
    tmp["My_measured"],
    tmp["Mz_measured"],
)).T

get_y = lambda tmp: np.vstack((
    tmp["Fx_applied"],
    tmp["Fy_applied"],
    tmp["Mz_applied"],
)).T

# ------------------------------------------------------------------------------
# Load Testing Data
# ------------------------------------------------------------------------------
X_test_svr = {}
X_test_rnn = {}
y_test = {}
for filename in test_files:
    data = loadmat(filename)
    data = data["training_data_struct"]
    X_temp = np.empty((0,6))
    y_temp = np.empty((0,3))
    for test in data.keys():
        X_temp = np.vstack((X_temp, get_X(data[test])))
        y_temp = np.vstack((y_temp, get_y(data[test])))

    X_test_svr[filename[:-4]] = X_temp[n_history:]
    X_test_rnn[filename[:-4]] = np.hstack([
        X_temp[i:len(X_temp)-n_history+i]
        for i in range(n_history+1)
    ])
    y_test[filename[:-4]] = y_temp[n_history:]

# ------------------------------------------------------------------------------
# Downsample from 400Hz to 200 Hz (realworld control frequency)
# ------------------------------------------------------------------------------
for filename in test_files:
    X_test_svr[filename[:-4]] = decimate(X_test_svr[filename[:-4]], 2, axis=0)
    X_test_rnn[filename[:-4]] = decimate(X_test_rnn[filename[:-4]], 2, axis=0)
    y_test[filename[:-4]] = decimate(y_test[filename[:-4]], 2, axis=0)

In [ ]:
test_database = "calibration_struct2"
y_ground = y_test[test_database]

# ------------------------------------------------------------------------------
# Comparison b/w RNN & SVR
# ------------------------------------------------------------------------------
model = prediction_model.BumperModel()

for j in range(X_test_rnn[test_database].shape[0]):
    x = np.array([X_test_rnn[test_database][j,:]])
    if j == 0:
        y_est_nn = [model.predict_nn(x)]
    else:
        y_est_nn = np.append(y_est_nn, [model.predict_nn(x)], axis=0)

for j in range(X_test_svr[test_database].shape[0]):
    x = np.array([X_test_svr[test_database][j,:]])
    if j == 0:
        y_est_svr = [model.predict_svr(x)]
    else:
        y_est_svr = np.append(y_est_svr, [model.predict_svr(x)], axis=0)

# ------------------------------------------------------------------------------
# Plots
# ------------------------------------------------------------------------------
out_name = ["Fx", "Fy", "Mz"]
colors = {
    "ground": "#d95f02",
    "rnn": "#1b9e77",
    "svr": "#7570b3",
}

plt.figure(test_database)
for i in range(len(out_name)):
    plt.subplot(3,1,i+1)
    plt.plot(y_ground[:, i], linewidth=3, color=colors["ground"])
    plt.plot(y_est_nn[:, i], color=colors["rnn"])
    plt.plot(y_est_svr[:, i], color=colors["svr"])
    if out_name[i] is "Fx":
        plt.ylabel("Fx [N]")
    elif out_name[i] is "Fy":
        plt.ylabel("Fy [N]")
    elif out_name[i] is "Mz":
        plt.ylabel("Mz [N-m]")
    plt.tight_layout()
    plt.xlim((100000, 130000))
    plt.xticks([])
    plt.show()
plt.legend(["Ground Truth", "RNN", "SVR"])

In [ ]:
# ------------------------------------------------------------------------------
# Timing Comparison
# ------------------------------------------------------------------------------
n = 1000
model = prediction_model.BumperModel()

x_single = np.array(X_test_rnn[test_database][1,:])
start = timer()
[model.predict_nn(x_single) for i in range(n)]
end = timer()
ellapsed = end - start
print("---- RNN ----")
print("Total Elapsed Time for {:d} calls = {:.5f} s".format(n, ellapsed))
print("Avg. Elapsed Time per call = {:.5f} s".format(ellapsed/n))

x_single = np.array([X_test_svr[test_database][1,:]])
start = timer()
[model.predict_svr(x_single) for i in range(n)]
end = timer()
ellapsed = end - start
print("---- SVR ----")
print("Total Elapsed Time for {:d} calls = {:.5f} s".format(n, ellapsed))
print("Avg. Elapsed Time per call = {:.5f} s".format(ellapsed/n))